In [1]:
# Импортируем необходимые библиотеки

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.callbacks import Callback, EarlyStopping
import math

# Импортируем загрузку модели
from keras.models import load_model

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.preprocessing import image
import numpy as np

In [30]:
TRAIN_DATA_DIR = 'C://Users//Legion 5//Desktop//Учёба//Стажки//Портфолио//Телеграмм бот для распознования Московских достопремичательностей//train'
VALIDATION_DATA_DIR = 'C://Users//Legion 5//Desktop//Учёба//Стажки//Портфолио//Телеграмм бот для распознования Московских достопремичательностей//valid'
TEST_DATA_DIR = 'C://Users//Legion 5//Desktop//Учёба//Стажки//Портфолио//Телеграмм бот для распознования Московских достопремичательностей//test'
NUM_CLASSES = 12
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [31]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
                                                    TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                                                        VALIDATION_DATA_DIR,
                                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                        batch_size=BATCH_SIZE,
                                                        shuffle=False,
                                                        seed = 12345,
                                                        class_mode='categorical')

test_generator = val_datagen.flow_from_directory(TEST_DATA_DIR,
                                                target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                batch_size=BATCH_SIZE,
                                                shuffle=False,
                                                seed = 12345,
                                                class_mode='categorical')

Found 3169 images belonging to 12 classes.
Found 800 images belonging to 12 classes.
Found 477 images belonging to 12 classes.


In [4]:
def model_maker():
    
    backbone = ResNet50(input_shape = (IMG_WIDTH,IMG_HEIGHT,3),
                         include_top = False,
                         weights = 'imagenet')
    
    backbone.trainable = False
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(2048, activation='relu'))
    #model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    #model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model

In [5]:
early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               patience = 3,
                               verbose=1,
                               restore_best_weights=True)

In [6]:
model = model_maker()
callbacks = early_stopping

model.compile(loss='categorical_crossentropy',
             optimizer= tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
             metrics=['acc'])

model.fit(train_generator,
         steps_per_epoch = 10,
         epochs=100,
         validation_data = validation_generator,
         validation_steps = 4,
         callbacks = callbacks,
         verbose=2)

Epoch 1/100
10/10 - 97s - loss: 2.0908 - acc: 0.3250 - val_loss: 2.0394 - val_acc: 0.3281 - 97s/epoch - 10s/step
Epoch 2/100
10/10 - 80s - loss: 1.3328 - acc: 0.6797 - val_loss: 1.2448 - val_acc: 0.7148 - 80s/epoch - 8s/step
Epoch 3/100
10/10 - 82s - loss: 0.7874 - acc: 0.8375 - val_loss: 0.6528 - val_acc: 0.8477 - 82s/epoch - 8s/step
Epoch 4/100
10/10 - 78s - loss: 0.5169 - acc: 0.8719 - val_loss: 0.4620 - val_acc: 0.8867 - 78s/epoch - 8s/step
Epoch 5/100
10/10 - 79s - loss: 0.4110 - acc: 0.9047 - val_loss: 0.3458 - val_acc: 0.8984 - 79s/epoch - 8s/step
Epoch 6/100
10/10 - 74s - loss: 0.3296 - acc: 0.9097 - val_loss: 0.3860 - val_acc: 0.8906 - 74s/epoch - 7s/step
Epoch 7/100
10/10 - 75s - loss: 0.3027 - acc: 0.9212 - val_loss: 0.3327 - val_acc: 0.9023 - 75s/epoch - 8s/step
Epoch 8/100
10/10 - 76s - loss: 0.2204 - acc: 0.9392 - val_loss: 0.2399 - val_acc: 0.9414 - 76s/epoch - 8s/step
Epoch 9/100
10/10 - 77s - loss: 0.1930 - acc: 0.9500 - val_loss: 0.2581 - val_acc: 0.9141 - 77s/epoch -

In [33]:
model.evaluate(test_generator)

8/8 [==============================] - 24s 3s/step - loss: 0.1941 - acc: 0.9455


[0.19405929744243622, 0.945492684841156]

In [17]:
img = image.load_img('C://Users//Legion 5//Desktop//Учёба//Стажки//Портфолио//Телеграмм бот для распознования Московских достопремичательностей//Мэрия.jpg', target_size=(224,224)) # Загружаем в нужном размере
img_array = image.img_to_array(img)               # Превращаем в массив
img_batch = np.expand_dims(img_array, axis = 0)   # Добавляем измерение (только когда одна фотка), иначе reshape
img_preprocessed = preprocess_input(img_batch)    # Нормализуем

predictions = model.predict(img_preprocessed)     # Предсказываем
print(predictions)
print(list(validation_generator.class_indices.keys())[np.argmax(predictions)])

1/1 [==============================] - 0s 112ms/step
[[2.7753647e-05 3.0694657e-05 2.4449355e-05 5.4127540e-06 9.9333161e-01
  9.8006385e-05 4.9328664e-05 1.1521391e-05 1.7098459e-05 5.7680372e-06
  5.7410912e-06 6.3926079e-03]]
Здание мэрии Москвы (бывший дом генерал-губернатора)


In [18]:
import pandas as pd
a = pd.DataFrame(data=[validation_generator.class_indices.keys(), predictions[0]]).T
a.sort_values(by=1, ascending = False)

,0,1
4,Здание мэрии Москвы (бывший дом генерал-губерн...,0.993332
11,ЦУМ,0.006393
5,Иверские (Воскресенские) ворота,0.000098
6,Мавзолей В. И. Ленина,0.000049
1,ГУМ,0.000031
0,Большой театр,0.000028
2,Государственный исторический музей,0.000024
8,Никольская улица,0.000017
7,Московский Кремль,0.000012
9,Собор Казанской иконы Божией Матери на Красной...,0.000006


In [19]:
model.save("moscow_sights_center(12).h5")